In [1]:
# standard library imports
import csv
import datetime as dt
import json
import os
import statistics
import time

# third-party imports
import numpy as np
import pandas as pd
import requests

# customisations - ensure tables show all columns
pd.set_option("max_columns", 100)

from config import api_key

We create a general get request function to pull from API

In [2]:
def get_request(url, parameters=None):
    """Return json-formatted response of a get request using optional parameters.
    
    Parameters
    ----------
    url : string
    parameters : {'parameter': 'value'}
        parameters to pass as part of get request
    
    Returns
    -------
    json_data
        json-formatted response (dict-like)
    """
    try:
        response = requests.get(url=url, params=parameters)
    except SSLError as s:
        print('SSL Error:', s)
        
        for i in range(5, 0, -1):
            print('\rWaiting... ({})'.format(i), end='')
            time.sleep(1)
        print('\rRetrying.' + ' '*10)
        
        # recusively try again
        return get_request(url, parameters)
    
    if response:
        return response.json()
    else:
        # response is none usually means too many requests. Wait and try again 
        print('No response, waiting 30 seconds...')
        time.sleep(30)
        print('Retrying.')
        return get_request(url, parameters)

Next we use SteamSpy's API to generate a list of games

In [5]:
url = "https://steamspy.com/api.php"
parameters = {"request": "all"}

# request 'all' from steam spy and parse into dataframe
json_data = get_request(url, parameters=parameters)
steam_spy_all = pd.DataFrame.from_dict(json_data, orient='index')

# generate sorted app_list from steamspy data
app_list = steam_spy_all[['appid', 'name']].sort_values('appid').reset_index(drop=True)



In [8]:
# export disabled to keep consistency across download sessions
app_list.to_csv('app_list_3.csv', index=False)

In [9]:
app_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37717 entries, 0 to 37716
Data columns (total 2 columns):
appid    37717 non-null int64
name     37717 non-null object
dtypes: int64(1), object(1)
memory usage: 589.5+ KB


In [11]:
# instead read from stored csv
app_list = pd.read_csv('app_list_3.csv')

# display first few rows
app_list.head()

,appid,name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force


In [12]:
def get_app_data(start, stop, parser, pause):
    """Return list of app data generated from parser.
    
    parser : function to handle request
    """
    app_data = []
    
    # iterate through each row of app_list, confined by start and stop
    for index, row in app_list[start:stop].iterrows():
        print('Current index: {}'.format(index), end='\r')
        
        appid = row['appid']
        name = row['name']

        # retrive app data for a row, handled by supplied parser, and append to list
        data = parser(appid, name)
        app_data.append(data)

        time.sleep(pause) # prevent overloading api with requests
    
    return app_data

In [13]:
def process_batches(parser, app_list, download_path, data_filename, index_filename,
                    columns, begin=0, end=-1, batchsize=100, pause=1):
    """Process app data in batches, writing directly to file.
    
    parser : custom function to format request
    app_list : dataframe of appid and name
    download_path : path to store data
    data_filename : filename to save app data
    index_filename : filename to store highest index written
    columns : column names for file
    
    Keyword arguments:
    
    begin : starting index (get from index_filename, default 0)
    end : index to finish (defaults to end of app_list)
    batchsize : number of apps to write in each batch (default 100)
    pause : time to wait after each api request (defualt 1)
    
    returns: none
    """
    print('Starting at index {}:\n'.format(begin))
    
    # by default, process all apps in app_list
    if end == -1:
        end = len(app_list) + 1
    
    # generate array of batch begin and end points
    batches = np.arange(begin, end, batchsize)
    batches = np.append(batches, end)
    
    apps_written = 0
    batch_times = []
    
    for i in range(len(batches) - 1):
            start_time = time.time()

            start = batches[i]
            stop = batches[i+1]

            app_data = get_app_data(start, stop, parser, pause)

            rel_path = os.path.join(download_path, data_filename)

            # writing app data to file
            with open(rel_path, 'a', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=columns, extrasaction='ignore')

                for j in range(3,0,-1):
                    print("\rAbout to write data, don't stop script! ({})".format(j), end='')
                    time.sleep(0.5)

                writer.writerows(app_data)
                print('\rExported lines {}-{} to {}.'.format(start, stop-1, data_filename), end=' ')

            apps_written += len(app_data)

            idx_path = os.path.join(download_path, index_filename)

            # writing last index to file
            with open(idx_path, 'w') as f:
                index = stop
                print(index, file=f)

            # logging time taken
            end_time = time.time()
            time_taken = end_time - start_time

            batch_times.append(time_taken)
            mean_time = statistics.mean(batch_times)

            est_remaining = (len(batches) - i - 2) * mean_time

            remaining_td = dt.timedelta(seconds=round(est_remaining))
            time_td = dt.timedelta(seconds=round(time_taken))
            mean_td = dt.timedelta(seconds=round(mean_time))

            print('Batch {} time: {} (avg: {}, remaining: {})'.format(i, time_td, mean_td, remaining_td))

    print('\nProcessing batches complete. {} apps written'.format(apps_written))

    

In [14]:
def reset_index(download_path, index_filename):
    """Reset index in file to 0."""
    rel_path = os.path.join(download_path, index_filename)
    
    with open(rel_path, 'w') as f:
        print(0, file=f)
        

def get_index(download_path, index_filename):
    """Retrieve index from file, returning 0 if file not found."""
    try:
        rel_path = os.path.join(download_path, index_filename)

        with open(rel_path, 'r') as f:
            index = int(f.readline())
    
    except FileNotFoundError:
        index = 0
        
    return index


def prepare_data_file(download_path, filename, index, columns):
    """Create file and write headers if index is 0."""
    if index == 0:
        rel_path = os.path.join(download_path, filename)

        with open(rel_path, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            writer.writeheader()

In [15]:
def parse_steam_request(appid, name):
    """Unique parser to handle data from Steam Store API.
    
    Returns : json formatted data (dict-like)
    """
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": appid}
    
    json_data = get_request(url, parameters=parameters)
    json_app_data = json_data[str(appid)]
    
    if json_app_data['success']:
        data = json_app_data['data']
    else:
        data = {'name': name, 'steam_appid': appid}
        
    return data


# Set file parameters
download_path = ''
steam_app_data = 'steam_app_data.csv'
steam_index = 'steam_index.txt'

steam_columns = [
    'type', 'name', 'steam_appid', 'required_age', 'is_free', 'controller_support',
    'dlc', 'detailed_description', 'about_the_game', 'short_description', 'fullgame',
    'supported_languages', 'header_image', 'website', 'pc_requirements', 'mac_requirements',
    'linux_requirements', 'legal_notice', 'drm_notice', 'ext_user_account_notice',
    'developers', 'publishers', 'demos', 'price_overview', 'packages', 'package_groups',
    'platforms', 'metacritic', 'reviews', 'categories', 'genres', 'screenshots',
    'movies', 'recommendations', 'achievements', 'release_date', 'support_info',
    'background', 'content_descriptors'
]

# Overwrites last index for demonstration (would usually store highest index so can continue across sessions)
reset_index(download_path, steam_index)

# Retrieve last index downloaded from file
index = get_index(download_path, steam_index)

# Wipe or create data file and write headers if index is 0
prepare_data_file(download_path, steam_app_data, index, steam_columns)

In [46]:
# Set end and chunksize for demonstration - remove to run through entire app list
process_batches(
    parser=parse_steam_request,
    app_list=app_list,
    download_path=download_path,
    data_filename=steam_app_data,
    index_filename=steam_index,
    columns=steam_columns,
    begin=35901,
    #end=35001,
    #batchsize=5
)

Starting at index 35901:

Exported lines 35901-36000 to steam_app_data.csv. Batch 0 time: 0:02:17 (avg: 0:02:17, remaining: 0:41:13)
Exported lines 36001-36100 to steam_app_data.csv. Batch 1 time: 0:02:16 (avg: 0:02:17, remaining: 0:38:46)
No response, waiting 30 seconds...
Retrying.
Exported lines 36101-36200 to steam_app_data.csv. Batch 2 time: 0:02:48 (avg: 0:02:27, remaining: 0:39:14)
Exported lines 36201-36300 to steam_app_data.csv. Batch 3 time: 0:02:18 (avg: 0:02:25, remaining: 0:36:11)
No response, waiting 30 seconds...
Retrying.
Exported lines 36301-36400 to steam_app_data.csv. Batch 4 time: 0:02:48 (avg: 0:02:29, remaining: 0:34:52)
Exported lines 36401-36500 to steam_app_data.csv. Batch 5 time: 0:02:18 (avg: 0:02:28, remaining: 0:31:58)
No response, waiting 30 seconds...
Retrying.
Exported lines 36501-36600 to steam_app_data.csv. Batch 6 time: 0:02:49 (avg: 0:02:31, remaining: 0:30:07)
Exported lines 36601-36700 to steam_app_data.csv. Batch 7 time: 0:02:17 (avg: 0:02:29, rem

In [47]:
df = pd.read_csv('steam_app_data.csv')

In [48]:
df['type'].value_counts()

game    37630
Name: type, dtype: int64

In [49]:
df.head()

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 999, 'final': 9...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 92459},NaN,"{'coming_soon': False, 'date': 'Nov 1, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 3491},NaN,"{'coming_soon': False, 'date': 'Apr 1, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'USD', 'initial': 499, 'final': 4...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2634},NaN,"{'coming_soon': False, 'date': 'May 1, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: 

In [32]:
index

0

In [54]:
def parse_steamspy_request(appid, name):
    """Parser to handle SteamSpy API data."""
    url = "https://steamspy.com/api.php"
    parameters = {"request": "appdetails", "appid": appid}
    json_data = get_request(url, parameters)
    return json_data
# set files and columns
download_path = ''
steamspy_data = 'steamspy_data.csv'
steamspy_index = 'steamspy_index.txt'
steamspy_columns = [
    'appid', 'name', 'developer', 'publisher', 'score_rank', 'positive',
    'negative', 'userscore', 'owners', 'average_forever', 'average_2weeks',
    'median_forever', 'median_2weeks', 'price', 'initialprice', 'discount',
    'languages', 'genre', 'ccu', 'tags'
]
reset_index(download_path, steamspy_index)
index = get_index(download_path, steamspy_index)
# Wipe data file if index is 0
prepare_data_file(download_path, steamspy_data, index, steamspy_columns)
process_batches(
    parser=parse_steamspy_request,
    app_list=app_list,
    download_path=download_path, 
    data_filename=steamspy_data,
    index_filename=steamspy_index,
    columns=steamspy_columns,
    begin=index,
    #end=20,
    #batchsize=5,
    pause=0.3
)


Starting at index 0:

Exported lines 0-99 to steamspy_data.csv. Batch 0 time: 0:00:55 (avg: 0:00:55, remaining: 5:45:40)
Exported lines 100-199 to steamspy_data.csv. Batch 1 time: 0:00:54 (avg: 0:00:55, remaining: 5:42:44)
Exported lines 200-299 to steamspy_data.csv. Batch 2 time: 0:00:53 (avg: 0:00:54, remaining: 5:39:15)
Exported lines 300-399 to steamspy_data.csv. Batch 3 time: 0:00:53 (avg: 0:00:54, remaining: 5:35:36)
Exported lines 400-499 to steamspy_data.csv. Batch 4 time: 0:00:53 (avg: 0:00:54, remaining: 5:33:27)
Exported lines 500-599 to steamspy_data.csv. Batch 5 time: 0:00:53 (avg: 0:00:54, remaining: 5:32:02)
Exported lines 600-699 to steamspy_data.csv. Batch 6 time: 0:00:54 (avg: 0:00:54, remaining: 5:31:27)
Exported lines 700-799 to steamspy_data.csv. Batch 7 time: 0:00:54 (avg: 0:00:54, remaining: 5:31:09)
Exported lines 800-899 to steamspy_data.csv. Batch 8 time: 0:00:53 (avg: 0:00:54, remaining: 5:29:39)
Exported lines 900-999 to steamspy_data.csv. Batch 9 time: 0:00

Exported lines 7800-7899 to steamspy_data.csv. Batch 78 time: 0:00:52 (avg: 0:00:53, remaining: 4:25:39)
Exported lines 7900-7999 to steamspy_data.csv. Batch 79 time: 0:00:52 (avg: 0:00:53, remaining: 4:24:39)
Exported lines 8000-8099 to steamspy_data.csv. Batch 80 time: 0:00:53 (avg: 0:00:53, remaining: 4:23:47)
Exported lines 8100-8199 to steamspy_data.csv. Batch 81 time: 0:00:53 (avg: 0:00:53, remaining: 4:22:51)
Exported lines 8200-8299 to steamspy_data.csv. Batch 82 time: 0:00:54 (avg: 0:00:53, remaining: 4:21:59)
Exported lines 8300-8399 to steamspy_data.csv. Batch 83 time: 0:00:53 (avg: 0:00:53, remaining: 4:21:06)
Exported lines 8400-8499 to steamspy_data.csv. Batch 84 time: 0:00:53 (avg: 0:00:53, remaining: 4:20:11)
Exported lines 8500-8599 to steamspy_data.csv. Batch 85 time: 0:00:53 (avg: 0:00:53, remaining: 4:19:16)
Exported lines 8600-8699 to steamspy_data.csv. Batch 86 time: 0:00:52 (avg: 0:00:53, remaining: 4:18:20)
Exported lines 8700-8799 to steamspy_data.csv. Batch 87

Exported lines 15500-15599 to steamspy_data.csv. Batch 155 time: 0:00:54 (avg: 0:00:53, remaining: 3:16:38)
Exported lines 15600-15699 to steamspy_data.csv. Batch 156 time: 0:00:53 (avg: 0:00:53, remaining: 3:15:44)
Exported lines 15700-15799 to steamspy_data.csv. Batch 157 time: 0:00:52 (avg: 0:00:53, remaining: 3:14:50)
Exported lines 15800-15899 to steamspy_data.csv. Batch 158 time: 0:00:54 (avg: 0:00:53, remaining: 3:13:57)
Exported lines 15900-15999 to steamspy_data.csv. Batch 159 time: 0:00:52 (avg: 0:00:53, remaining: 3:13:03)
Exported lines 16000-16099 to steamspy_data.csv. Batch 160 time: 0:00:53 (avg: 0:00:53, remaining: 3:12:10)
Exported lines 16100-16199 to steamspy_data.csv. Batch 161 time: 0:00:51 (avg: 0:00:53, remaining: 3:11:14)
Exported lines 16200-16299 to steamspy_data.csv. Batch 162 time: 0:00:50 (avg: 0:00:53, remaining: 3:10:18)
Exported lines 16300-16399 to steamspy_data.csv. Batch 163 time: 0:00:51 (avg: 0:00:53, remaining: 3:09:21)
Exported lines 16400-16499 t

Exported lines 23100-23199 to steamspy_data.csv. Batch 231 time: 0:00:52 (avg: 0:00:53, remaining: 2:09:11)
Exported lines 23200-23299 to steamspy_data.csv. Batch 232 time: 0:00:52 (avg: 0:00:53, remaining: 2:08:17)
Exported lines 23300-23399 to steamspy_data.csv. Batch 233 time: 0:00:53 (avg: 0:00:53, remaining: 2:07:24)
Exported lines 23400-23499 to steamspy_data.csv. Batch 234 time: 0:00:54 (avg: 0:00:53, remaining: 2:06:31)
Exported lines 23500-23599 to steamspy_data.csv. Batch 235 time: 0:00:53 (avg: 0:00:53, remaining: 2:05:38)
Exported lines 23600-23699 to steamspy_data.csv. Batch 236 time: 0:00:54 (avg: 0:00:53, remaining: 2:04:45)
Exported lines 23700-23799 to steamspy_data.csv. Batch 237 time: 0:00:53 (avg: 0:00:53, remaining: 2:03:52)
Exported lines 23800-23899 to steamspy_data.csv. Batch 238 time: 0:00:55 (avg: 0:00:53, remaining: 2:03:01)
Exported lines 23900-23999 to steamspy_data.csv. Batch 239 time: 0:00:54 (avg: 0:00:53, remaining: 2:02:08)
Exported lines 24000-24099 t

Exported lines 30700-30799 to steamspy_data.csv. Batch 307 time: 0:00:53 (avg: 0:00:53, remaining: 1:02:02)
Exported lines 30800-30899 to steamspy_data.csv. Batch 308 time: 0:00:52 (avg: 0:00:53, remaining: 1:01:08)
Exported lines 30900-30999 to steamspy_data.csv. Batch 309 time: 0:00:52 (avg: 0:00:53, remaining: 1:00:15)
Exported lines 31000-31099 to steamspy_data.csv. Batch 310 time: 0:00:54 (avg: 0:00:53, remaining: 0:59:22)
Exported lines 31100-31199 to steamspy_data.csv. Batch 311 time: 0:00:53 (avg: 0:00:53, remaining: 0:58:29)
Exported lines 31200-31299 to steamspy_data.csv. Batch 312 time: 0:00:53 (avg: 0:00:53, remaining: 0:57:35)
Exported lines 31300-31399 to steamspy_data.csv. Batch 313 time: 0:00:53 (avg: 0:00:53, remaining: 0:56:42)
Exported lines 31400-31499 to steamspy_data.csv. Batch 314 time: 0:00:52 (avg: 0:00:53, remaining: 0:55:49)
Exported lines 31500-31599 to steamspy_data.csv. Batch 315 time: 0:00:53 (avg: 0:00:53, remaining: 0:54:56)
Exported lines 31600-31699 t

In [55]:
df1= pd.read_csv('steamspy_data.csv')

In [57]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37717 entries, 0 to 37716
Data columns (total 20 columns):
appid              37717 non-null int64
name               37708 non-null object
developer          37530 non-null object
publisher          37516 non-null object
score_rank         46 non-null float64
positive           37717 non-null int64
negative           37717 non-null int64
userscore          37717 non-null int64
owners             37717 non-null object
average_forever    37717 non-null int64
average_2weeks     37717 non-null int64
median_forever     37717 non-null int64
median_2weeks      37717 non-null int64
price              37688 non-null float64
initialprice       37695 non-null float64
discount           37695 non-null float64
languages          37629 non-null object
genre              37494 non-null object
ccu                37717 non-null int64
tags               37717 non-null object
dtypes: float64(4), int64(9), object(7)
memory usage: 5.8+ MB


In [59]:
df1.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,10,Counter-Strike,Valve,Valve,NaN,159745,4173,0,"10,000,000 .. 20,000,000",21211,1237,172,81,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,15194,"{'Action': 5314, 'FPS': 4718, 'Multiplayer': 3..."
1,20,Team Fortress Classic,Valve,Valve,NaN,4335,771,0,"2,000,000 .. 5,000,000",798,0,11,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,123,"{'Action': 731, 'FPS': 297, 'Multiplayer': 251..."
2,30,Day of Defeat,Valve,Valve,NaN,4325,482,0,"5,000,000 .. 10,000,000",894,7,18,7,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Spain",Action,140,"{'FPS': 777, 'World War II': 242, 'Multiplayer..."
3,40,Deathmatch Classic,Valve,Valve,NaN,1592,345,0,"5,000,000 .. 10,000,000",1200,0,8,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,5,"{'Action': 624, 'FPS': 135, 'Classic': 103, 'M..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,NaN,8907,477,0,"5,000,000 .. 10,000,000",1316,391,281,391,499.0,499.0,0.0,"English, French, German, Korean",Action,104,"{'FPS': 869, 'Action': 310, 'Classic': 237, 'S..."
